# Content Based Recommendation

1. Membuat Metada Per-Item

In [1]:
import joblib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re

from sklearn.feature_extraction.text import CountVectorizer

import pymongo

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

from wordcloud import WordCloud,STOPWORDS 

# SETTING
plt.style.use('ggplot')
#display maximum column and rows
pd.options.display.max_columns = 999
pd.options.display.min_rows = 999
pd.options.display.max_colwidth = 999
stopword = STOPWORDS
stopword.add('im')
stopword.add('youre')


red = '#ef4b4b'
black = '#584b42'
green = '#a0cc78'
yellow_ = '#f6d365'
blue_ = '#49beb7'

#INIT


#METHODS
def sep():
    print(100*'-')

def connectMongoDB(dbname):
    client = pymongo.MongoClient('localhost', 27017)
    db = client[dbname]
    return db

dbname = 'clothing_fit'
db = connectMongoDB(dbname)

In [3]:
rent = list(db['rentCollection'].find())
df_rent = pd.DataFrame(rent)
df_rent

# MENGECEK NULL
print("banyak data : ", len(df_rent))
print("columns : ", df_rent.columns)

df_rent.info()


banyak data :  20027
columns :  Index(['_id', 'fit', 'user_id', 'bust size', 'item_id', 'weight', 'rating',
       'rented for', 'review_text', 'body type', 'review_summary', 'category',
       'height', 'size', 'age', 'review_date', 'sentiment_', 'height_cm',
       'weight_kg', 'age_', 'review_date_', 'sentiment_summary',
       'sentiment_text'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20027 entries, 0 to 20026
Data columns (total 23 columns):
_id                  20027 non-null object
fit                  20027 non-null object
user_id              20027 non-null object
bust size            18126 non-null object
item_id              20027 non-null object
weight               16866 non-null object
rating               20027 non-null float64
rented for           20027 non-null object
review_text          20027 non-null object
body type            18517 non-null object
review_summary       20027 non-null object
category             20027 non-null object


In [4]:
# SELEKSI KOLOM yang Mungkin Akan diproses

cols_needed = ['_id', 'fit', 'user_id', 'bust size', 'item_id', 'rating',
       'rented for', 'body type', 'category', 'size', 'sentiment_', 'height_cm',
       'weight_kg', 'age_', 'review_date_', 'sentiment_summary',
       'sentiment_text']
df_rent = df_rent[cols_needed].copy()
df_rent.head()
df_rent.isnull().sum()

_id                     0
fit                     0
user_id                 0
bust size            1901
item_id                 0
rating                  0
rented for              0
body type            1510
category                0
size                    0
sentiment_              0
height_cm               0
weight_kg            3161
age_                    0
review_date_            0
sentiment_summary       0
sentiment_text          0
dtype: int64

In [5]:
df_rent = df_rent.dropna(subset=['body type', 'weight_kg'])
len(df_rent)
df_rent['rented for'].unique()

array(['vacation', 'other', 'formal affair', 'wedding', 'date',
       'everyday', 'party', 'work'], dtype=object)

In [6]:
df_rent['id'] = range(15982)
df_rent.isnull().sum()
df_rent = df_rent.set_index('id')
df_rent.head()

,_id,fit,user_id,bust size,item_id,rating,rented for,body type,category,size,sentiment_,height_cm,weight_kg,age_,review_date_,sentiment_summary,sentiment_text
id,,,,,,,,,,,,,,,,,
0,5e2ab8ce9f84980caca61bf7,fit,420272,34d,2260466,10.0,vacation,hourglass,romper,14,positive,173.0,62.0,28.0,2016-04-20,positive,positive
1,5e2ab8ce9f84980caca61bf8,fit,273551,34b,153475,10.0,other,straight & narrow,gown,12,positive,168.0,60.0,36.0,2013-06-18,positive,positive
2,5e2ab8ce9f84980caca61bfa,fit,909926,34c,126335,8.0,formal affair,pear,dress,8,positive,165.0,61.0,34.0,2014-02-12,positive,positive
3,5e2ab8ce9f84980caca61bfb,fit,151944,34b,616682,10.0,wedding,athletic,gown,12,positive,175.0,66.0,27.0,2016-09-26,positive,positive
4,5e2ab8ce9f84980caca61bfc,fit,734848,32b,364092,8.0,date,athletic,dress,8,positive,173.0,63.0,45.0,2016-04-30,neutral,positive


## One Hot transform untuk kolomr rented for

In [56]:
# TRANSFORMING one hot untuk rented for
encRentfor = preprocessing.OneHotEncoder()

print('banyak data : ', df_rent.shape)
encRentfor.fit(df_rent[['rented for']])
x = encRentfor.transform(df_rent[['rented for']]).toarray()
print(encRentfor.categories_)
print(x)
cols = encRentfor.categories_[0]
df_rentfor = pd.DataFrame(x, columns = cols)
print(df_rentfor.shape)
print(df_rent.shape)

df_rentfor.isnull().sum()
# df_rentfor.head()
# df_rent.head()

banyak data :  (15982, 28)
[array(['date', 'everyday', 'formal affair', 'other', 'party', 'vacation',
       'wedding', 'work'], dtype=object)]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(15982, 8)
(15982, 28)


date             0
everyday         0
formal affair    0
other            0
party            0
vacation         0
wedding          0
work             0
dtype: int64

In [8]:
#encRentfor.transform([['other']]).toarray()[0].tolist()

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [53]:
# eksporting

joblib.dump(encRentfor, 'onehot_rentfor')

['onehot_rentfor']

In [10]:
# RUN SEKALI SAJA!

print("banyak data : ", df_rentfor.shape)
print(df_rentfor.shape)
print(df_rent.shape)
df_rent = pd.concat([df_rent, df_rentfor], axis=1) ###########!!! concat 1 kali
df_rent.head()
print(df_rentfor.shape)
print(df_rent.shape)

banyak data :  (15982, 8)
(15982, 8)
(15982, 17)
(15982, 8)
(15982, 25)


In [11]:
df_rent.isnull().sum()

_id                    0
fit                    0
user_id                0
bust size            679
item_id                0
rating                 0
rented for             0
body type              0
category               0
size                   0
sentiment_             0
height_cm              0
weight_kg              0
age_                   0
review_date_           0
sentiment_summary      0
sentiment_text         0
date                   0
everyday               0
formal affair          0
other                  0
party                  0
vacation               0
wedding                0
work                   0
dtype: int64

In [12]:
print(len(df_rent))
print(df_rent.columns)
df_rent.isnull().sum()
df_rentfor.isnull().sum()

15982
Index(['_id', 'fit', 'user_id', 'bust size', 'item_id', 'rating', 'rented for',
       'body type', 'category', 'size', 'sentiment_', 'height_cm', 'weight_kg',
       'age_', 'review_date_', 'sentiment_summary', 'sentiment_text', 'date',
       'everyday', 'formal affair', 'other', 'party', 'vacation', 'wedding',
       'work'],
      dtype='object')


date             0
everyday         0
formal affair    0
other            0
party            0
vacation         0
wedding          0
work             0
dtype: int64

## Melihat kondisi jumlah fit, large, dan small

In [13]:
xx = df_rent.groupby(['fit','item_id','size']).mean()

# melihat kondisi masing2 item
df_fit = xx.loc[('fit')]
df_small = xx.loc[('small')]
df_large = xx.loc[('large')]

print("fit : ", len(df_fit))
print("small : ", len(df_small))
print("large : ", len(df_large))
# plt.figure(figsize=(15,15))
# sb.heatmap(xx.corr(), annot=True)

fit :  7343
small :  1807
large :  1703


## One Hot transform untuk Kolom Fit

In [14]:
# TRANSFORMING one hot untuk fit

encLabelfit = preprocessing.OneHotEncoder()
encLabelfit.fit(df_rent[['fit']])
fit_array = encLabelfit.transform(df_rent[['fit']]).toarray()
print(encLabelfit.categories_[0])
fit_array

df_fit = pd.DataFrame(fit_array, columns = encLabelfit.categories_[0] )
df_fit.head()

['fit' 'large' 'small']


,fit,large,small
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [15]:
## Eksport

joblib.dump(encLabelfit, 'onehot_fit')

['onehot_fit']

In [16]:
#menyamakan index agar mudah untuk digabung

df_fit = pd.DataFrame(df_fit, index = df_rent.index)
df_fit.index
df_rent.index

counter=0
for i in df_fit.index:
    if i != df_rent.index[counter]:
        print("beda")
    counter += 1

In [17]:
#menggabung hasil one hot kolom fit

print((df_fit.shape))
print((df_rent.shape))

df_rent = pd.concat([df_rent, df_fit], axis=1)
df_rent.head()

print((df_fit.shape))
print((df_rent.shape))

(15982, 3)
(15982, 25)
(15982, 3)
(15982, 28)


In [18]:
df_rent.isnull().sum()

_id                    0
fit                    0
user_id                0
bust size            679
item_id                0
rating                 0
rented for             0
body type              0
category               0
size                   0
sentiment_             0
height_cm              0
weight_kg              0
age_                   0
review_date_           0
sentiment_summary      0
sentiment_text         0
date                   0
everyday               0
formal affair          0
other                  0
party                  0
vacation               0
wedding                0
work                   0
fit                    0
large                  0
small                  0
dtype: int64

# transform metadata content matrix dengan index (item_id x size)
karena masing2 baju dengan ukuran berbeda menandakan kecocokan item yang berbeda

In [19]:
df_rent.isnull().sum()

_id                    0
fit                    0
user_id                0
bust size            679
item_id                0
rating                 0
rented for             0
body type              0
category               0
size                   0
sentiment_             0
height_cm              0
weight_kg              0
age_                   0
review_date_           0
sentiment_summary      0
sentiment_text         0
date                   0
everyday               0
formal affair          0
other                  0
party                  0
vacation               0
wedding                0
work                   0
fit                    0
large                  0
small                  0
dtype: int64

In [20]:
df_metadata = df_rent.groupby(['item_id','size']).mean()
df_metadata['item_unique'] = df_metadata.index
print(df_metadata.shape)
df_metadata.head()

(9377, 16)


rating  height_cm  weight_kg  age_  date  everyday  \
item_id size                                                       
1000254 4       10.0      157.0       52.0  26.0   0.0       0.0   
        12      10.0      163.0       59.0  45.0   0.0       0.0   
        16      10.0      165.0       64.0  27.0   0.0       0.0   
1000270 1       10.0      157.0       49.0  41.0   0.0       0.0   
        4       10.0      165.0       56.0  30.5   0.0       0.0   

              formal affair  other  party  vacation  wedding  work  fit  \
item_id size                                                              
1000254 4               0.0    0.0    0.0       0.0      1.0   0.0  1.0   
        12              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
        16              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
1000270 1               0.0    1.0    0.0       0.0      0.0   0.0  1.0   
        4               0.0    0.0    1.0       0.0      0.0   0.0  0.5   

              large  small    item_unique  
item_id size                               
1000254 4       0.0    0.0   (1000254, 4)  
        12      0.0    1.0  (1000254, 12)  
        16      0.0    1.0  (1000254, 16)  
1000270 1       0.0    0.0   (1000270, 1)  
        4       0.0    0.5   (1000270, 4)

In [21]:
df_metadata['vote_count'] = df_rent.groupby(['item_id', 'size']).count()['rating']
df_metadata.head()

rating  height_cm  weight_kg  age_  date  everyday  \
item_id size                                                       
1000254 4       10.0      157.0       52.0  26.0   0.0       0.0   
        12      10.0      163.0       59.0  45.0   0.0       0.0   
        16      10.0      165.0       64.0  27.0   0.0       0.0   
1000270 1       10.0      157.0       49.0  41.0   0.0       0.0   
        4       10.0      165.0       56.0  30.5   0.0       0.0   

              formal affair  other  party  vacation  wedding  work  fit  \
item_id size                                                              
1000254 4               0.0    0.0    0.0       0.0      1.0   0.0  1.0   
        12              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
        16              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
1000270 1               0.0    1.0    0.0       0.0      0.0   0.0  1.0   
        4               0.0    0.0    1.0       0.0      0.0   0.0  0.5   

              large  small    item_unique  vote_count  
item_id size                                           
1000254 4       0.0    0.0   (1000254, 4)           1  
        12      0.0    1.0  (1000254, 12)           1  
        16      0.0    1.0  (1000254, 16)           1  
1000270 1       0.0    0.0   (1000270, 1)           1  
        4       0.0    0.5   (1000270, 4)           2

## Weighted Rating 
### $$ WR = (\frac {v}{v+m} . R)+(\frac {m}{v+m}.C) $$


#### - R = rata-rata rating tiap item
#### - v = jumlah vote untuk masing2 item
#### - C = mean vote dari semua data yang tersedia
#### - m = minimum votes yang memenuhi syarat (treshold : persentil ke-90)


<hr>

In [22]:
# Calculate C (mean vote dari semua data yang tersedia)

C = df_metadata['rating'].mean()
print(C)
# -------------------------------------------------------------------------
# Calculate the minimum number of votes required to be in the chart, m 
## persentil ke 90 dari angka vote keseluruhan data

m = df_metadata['vote_count'].quantile(0.90)
print(m)

# -------------------------------------------------------------------------
# Function that computes the weighted rating of each clothes

def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['rating']
    
    # Calculation based on the IMDB formula
    # jumlah Nvote * rating
    return (v/(v+m) * R) + (m/(m+v) * C)



9.072789099622456
3.0


In [23]:
# -------------------------------------------------------------------------
# Define a new feature 'score' and calculate its value with `weighted_rating()`

df_metadata['score'] = df_metadata.apply(weighted_rating, axis=1)
df_metadata.head()
# q_clothes.shape
# q_clothes

rating  height_cm  weight_kg  age_  date  everyday  \
item_id size                                                       
1000254 4       10.0      157.0       52.0  26.0   0.0       0.0   
        12      10.0      163.0       59.0  45.0   0.0       0.0   
        16      10.0      165.0       64.0  27.0   0.0       0.0   
1000270 1       10.0      157.0       49.0  41.0   0.0       0.0   
        4       10.0      165.0       56.0  30.5   0.0       0.0   

              formal affair  other  party  vacation  wedding  work  fit  \
item_id size                                                              
1000254 4               0.0    0.0    0.0       0.0      1.0   0.0  1.0   
        12              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
        16              0.0    0.0    1.0       0.0      0.0   0.0  0.0   
1000270 1               0.0    1.0    0.0       0.0      0.0   0.0  1.0   
        4               0.0    0.0    1.0       0.0      0.0   0.0  0.5   

              large  small    item_unique  vote_count     score  
item_id size                                                     
1000254 4       0.0    0.0   (1000254, 4)           1  9.304592  
        12      0.0    1.0  (1000254, 12)           1  9.304592  
        16      0.0    1.0  (1000254, 16)           1  9.304592  
1000270 1       0.0    0.0   (1000270, 1)           1  9.304592  
        4       0.0    0.5   (1000270, 4)           2  9.443673

In [24]:
# mengubah index dataframe menjadi tuple item_id, size

df_metadata['item_id_size'] = df_metadata['item_unique'].apply(lambda x : x[0] +","+str(int(x[1])))
# df_metadata['item_unique'] = df_metadata['item_id_size']
df_metadata.head()
print(len(df_metadata))
df_metadata = df_metadata.set_index('item_unique', inplace=False)
df_metadata.head()

9377


,rating,height_cm,weight_kg,age_,date,everyday,formal affair,other,party,vacation,wedding,work,fit,large,small,vote_count,score,item_id_size
item_unique,,,,,,,,,,,,,,,,,,
"(1000254, 4)",10.0,157.0,52.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,9.304592,"1000254,4"
"(1000254, 12)",10.0,163.0,59.0,45.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,9.304592,"1000254,12"
"(1000254, 16)",10.0,165.0,64.0,27.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,9.304592,"1000254,16"
"(1000270, 1)",10.0,157.0,49.0,41.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,9.304592,"1000270,1"
"(1000270, 4)",10.0,165.0,56.0,30.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.5,2,9.443673,"1000270,4"


In [25]:
# ada 9377 item unik (id x size)
df_metadata['counter'] = range(len(df_metadata))
df_metadata.tail(3)

,rating,height_cm,weight_kg,age_,date,everyday,formal affair,other,party,vacation,wedding,work,fit,large,small,vote_count,score,item_id_size,counter
item_unique,,,,,,,,,,,,,,,,,,,
"(999837, 8)",9.0,162.5,55.0,34.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,9.043673,"999837,8",9374
"(999837, 12)",9.6,161.4,60.2,28.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.4,0.0,0.6,5,9.402296,"999837,12",9375
"(999837, 16)",9.0,168.0,60.5,31.5,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.5,0.0,0.5,2,9.043673,"999837,16",9376


In [26]:
# mengeksport metadata item (content reference)

df_metadata.to_pickle('items_metadata')

# Membentuk DF yang hanya berisi fitur yang akan diuji dalam rekomendasi

### ['score', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday','formal affair', 'other', 'party', 'vacation', 'wedding', 'work', 'fit','large', 'small']

In [27]:
cols = ['score', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday','formal affair', 'other', 'party', 'vacation', 'wedding', 'work', 'fit','large', 'small']
# dari kolom ini, input user adalah
df = df_metadata[cols].copy()


In [73]:
df.isnull().sum()


# Menghitung Similarity

In [57]:
# Import cosine
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(df, df)

df_cosine_sim = pd.DataFrame(cosine_sim)
df_cosine_sim.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,...,8878,8879,8880,8881,8882,8883,8884,8885,8886,8887,8888,8889,8890,8891,8892,8893,8894,8895,8896,8897,8898,8899,8900,8901,8902,8903,8904,8905,8906,8907,8908,8909,8910,8911,8912,8913,8914,8915,8916,8917,8918,8919,8920,8921,8922,8923,8924,8925,8926,8927,8928,8929,8930,8931,8932,8933,8934,8935,8936,8937,8938,8939,8940,8941,8942,8943,8944,8945,8946,8947,8948,8949,8950,8951,8952,8953,8954,8955,8956,8957,8958,8959,8960,8961,8962,8963,8964,8965,8966,8967,8968,8969,8970,8971,8972,8973,8974,8975,8976,8977,8978,8979,8980,8981,8982,8983,8984,8985,8986,8987,8988,8989,8990,8991,8992,8993,8994,8995,8996,8997,8998,8999,9000,9001,9002,9003,9004,9005,9006,9007,9008,9009,9010,9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030,9031,9032,9033,9034,9035,9036,9037,9038,9039,9040,9041,9042,9043,9044,9045,9046,9047,9048,9049,9050,9051,9052,9053,9054,9055,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065,9066,9067,9068,9069,9070,9071,9072,9073,9074,9075,9076,9077,9078,9079,9080,9081,9082,9083,9084,9085,9086,9087,9088,9089,9090,9091,9092,9093,9094,9095,9096,9097,9098,9099,9100,9101,9102,9103,9104,9105,9106,9107,9108,9109,9110,9111,9112,9113,9114,9115,9116,9117,9118,9119,9120,9121,9122,9123,9124,9125,9126,9127,9128,9129,9130,9131,9132,9133,9134,9135,9136,9137,9138,9139,9140,9141,9142,9143,9144,9145,9146,9147,9148,9149,9150,9151,9152,9153,9154,9155,9156,9157,9158,9159,9160,9161,9162,9163,9164,9165,9166,9167,9168,9169,9170,9171,9172,9173,9174,9175,9176,9177,9178,9179,9180,9181,9182,9183,9184,9185,9186,9187,9188,9189,9190,9191,9192,9193,9194,9195,9196,9197,9198,9199,9200,9201,9202,9203,9204,9205,9206,9207,9208,9209,9210,9211,9212,9213,9214,9215,9216,9217,9218,9219,9220,9221,9222,9223,9224,9225,9226,9227,9228,9229,9230,9231,9232,9233,9234,9235,9236,9237,9238,9239,9240,9241,9242,9243,9244,9245,9246,9247,9248,9249,9250,9251,9252,9253,9254,9255,9256,9257,9258,9259,9260,9261,9262,9263,9264,9265,9266,9267,9268,9269,9270,9271,9272,9273,9274,9275,9276,9277,9278,9279,9280,9281,9282,9283,9284,9285,9286,9287,9288,9289,9290,9291,9292,9293,9294,9295,9296,9297,9298,9299

In [30]:
print(df.shape)
print(cosine_sim.shape)

(9377, 15)
(9377, 9377)


In [31]:
id = 999837
size = 12

# Steps content based

id_size = str(id)+','+str(size)
selected = df_metadata[df_metadata['item_id_size'] == id_size]

# Get the index of the clothes that matches the id and size
index = selected['counter']
print(index)
df_metadata.iloc[index]

# Get the pairwsie similarity scores of all clothes with that clothes
sim_scores = list(enumerate(cosine_sim[index][0]))


# # Sort the clothes based on the similarity scores & pearson
sim_scores_sort = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# # print(sim_scores_sort)
# # Get the scores of the 10 most similar clothes
sim_scores_top = sim_scores_sort[1:11]
# print(sim_scores_top)

#  # Get the clothes indices
clothes_indices = [i[0] for i in sim_scores_top]
print(clothes_indices)

item_unique
(999837, 12)    9375
Name: counter, dtype: int32
[2405, 7250, 2333, 2767, 1430, 1699, 3109, 16, 961, 9031]


In [32]:
# hasil yang didapatkan

counter=0
for i in clothes_indices:
    counter+=1
    print(counter)
    print(df_metadata[df_metadata['counter'] == i]['score'])

1
item_unique
(1478427, 12)    9.443673
Name: score, dtype: float64
2
item_unique
(397303, 12)    8.804592
Name: score, dtype: float64
3
item_unique
(1463604, 20)    9.304592
Name: score, dtype: float64
4
item_unique
(154002, 17)    9.443673
Name: score, dtype: float64
5
item_unique
(1314701, 14)    9.304592
Name: score, dtype: float64
6
item_unique
(1352656, 8)    9.304592
Name: score, dtype: float64
7
item_unique
(1618192, 8)    9.304592
Name: score, dtype: float64
8
item_unique
(1001122, 12)    9.304592
Name: score, dtype: float64
9
item_unique
(1238932, 8)    9.402296
Name: score, dtype: float64
10
item_unique
(909677, 16)    8.804592
Name: score, dtype: float64


In [33]:
df.shape

(9377, 15)

In [34]:
df.columns

Index(['score', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday',
       'formal affair', 'other', 'party', 'vacation', 'wedding', 'work', 'fit',
       'large', 'small'],
      dtype='object')

# karena kolom yang dipakai cukup panjang 15 kolom, konten akan diubah menjadi embedding dengan di PCA

In [69]:
# PCA / EMBEDDING CONTENT

df_embed = df.copy()
## dari 15 jadi 3
## 15 -> ['score', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday','formal affair',
#        'other', 'party', 'vacation', 'wedding', 'work', 'fit', 'large',
#        'small']
## 3 -> [ embed1, embed2, embed3 ] 
# explained_variance_ratio_ -> (0.994672628366279)

from sklearn.decomposition import PCA
pca2 = PCA(n_components=2)
pca2.fit(df_embed)
df_embed2 = pca2.transform(df_embed)
print("EVR 2 comp : ",pca2.explained_variance_ratio_.sum())
df_embed2

pca4 = PCA(n_components=4)
pca4.fit(df_embed)
df_embed4 = pca4.transform(df_embed)
print("EVR 4 comp : ",pca4.explained_variance_ratio_.sum())
df_embed4

pca = PCA(n_components=3)
pca.fit(df_embed)
df_embed = pca.transform(df_embed)
print("EVR 3 comp : ",pca.explained_variance_ratio_.sum())
df_embed



EVR 2 comp :  0.8173555357718739
EVR 4 comp :  0.9955002871630472
EVR 3 comp :  0.9941869468407123


array([[-13.29453452,  -6.81039504,   4.9731126 ],
       [ -3.01218534,  10.98207183,   0.08118066],
       [  0.24808184,  -7.52016869,  -0.03654709],
       ...,
       [ -8.23650163,   0.49660662,   0.03613582],
       [ -4.1349154 ,  -5.9273446 ,   2.53851704],
       [ -1.94452288,  -2.88572318,  -3.93278307]])

### explained variance ratio dari PCA 15 menjadi 3-4 ke atas cukup baik, 0.994 ke atas, sedangkan jika dikurangi jadi 2 akan turun banyak menjadi 0.81

    #### Sehingga diambil PCA menjadi 3 

In [36]:
# Eksport PCA decomposer

import joblib


# MENYIMPAN DENGAN JOBLIB
joblib.dump(pca, 'PCA_15_to_3')
joblib.dump(df_embed, 'items_embed')

['items_embed']

In [37]:
# membuat matrix similarity

df_embed.shape
cos_score_one = cosine_similarity(df_embed,[[-13.29475289,  -6.8098584 ,   4.97293176]])
cos_score_one.shape

cosine_sim_matrix = cosine_similarity(df_embed, df_embed)

df_cosine_sim_matrix = pd.DataFrame(cosine_sim_matrix)

In [38]:
df_cosine_sim_matrix.to_pickle('content_similarity')

In [39]:
# ['score', 'height_cm', 'weight_kg', 'age_', 'date', 'formal affair',
#        'other', 'party', 'vacation', 'wedding', 'work', 'fit', 'large',
#        'small']
# UJI INPUT


skor = 10
tinggi = 155
berat = 65
umur = 22
date = 0
everyday = 0
formal = 0
other = 0
party = 0
vacation = 0
wedding = 1
work = 0
fit = 1
large = 0
small = 0

my_input = [skor, tinggi, berat, umur, date, everyday,
         formal, other, party, vacation, wedding, 
         work, fit, large, small]



In [40]:
# LOAD PCA

my_pca = joblib.load('PCA_15_to_3')
input_embedding = my_pca.transform([my_input])

In [41]:
#
cos_score_one = cosine_similarity(df_embed,input_embedding)
cos_score_one

array([[ 0.61891305],
       [-0.70137701],
       [ 0.75349124],
       ...,
       [ 0.06040713],
       [ 0.85679592],
       [-0.02390017]])

In [42]:
# JIKA REFERENSI BERUPA ITEM YANG SUDAH ADA (MASUKKAN ID_SIZE)

id_size = ('999837', 12)
# id_size = None
n = 5

if (id_size) :
    id_size = str(id_size[0])+','+str(id_size[1])
    selected = df_metadata[df_metadata['item_id_size'] == id_size]

    # Get the index of the clothes that matches the id and size
    index = selected['counter']
    print("your previous favs : ",index[0])
    df_metadata.iloc[index]

    # Get the pairwsie similarity scores of all clothes with that clothes
    sim_scores = list(enumerate(cosine_sim_matrix[index][0]))
else : 
    input_embedding = my_pca.transform([my_input])
    sim_scores = enumerate(cosine_similarity(df_embed, input_embedding))

# # Sort the clothes based on the similarity scores & pearson
sim_scores_sort = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# # print(sim_scores_sort)
# # Get the scores of the 10 most similar clothes
sim_scores_top = sim_scores_sort[1:n+1]
# print(sim_scores_top)
sep()

#  # Get the clothes indices
clothes_indices = [i[0] for i in sim_scores_top]
print("Based on Your Previous : ")
print(clothes_indices)
# print(clothes_indices2)

your previous favs :  9375
----------------------------------------------------------------------------------------------------
Based on Your Previous : 
[8374, 1670, 8589, 7787, 277]


In [43]:
def getItemInCounter(indices):
    result = pd.DataFrame()
    for i in indices:
        tem = df_metadata[df_metadata['counter'] == i]
        result = pd.concat([result,tem])
    return result

In [44]:
clothes_indices

[8374, 1670, 8589, 7787, 277]

In [45]:
hasil_rekom = getItemInCounter(clothes_indices)
hasil_rekom = hasil_rekom.values

In [46]:

hasil_rekom[0][2]

59.0

# Evaluation

In [84]:
id_item_all = df.index

res = []
for i in range(len(id_item_all)):
#     print(id_item_all[i])
    id_size = id_item_all[i]
    id_size = str(id_size[0])+','+str(id_size[1])
    selected = df_metadata[df_metadata['item_id_size'] == id_size]

    # Get the index of the clothes that matches the id and size
    index = selected['counter']
#     print("your previous favs : ",index[0])
    df_metadata.iloc[index]

    # Get the pairwsie similarity scores of all clothes with that clothes
    sim_scores = list(enumerate(cosine_sim_matrix[index][0]))
    
    # # Sort the clothes based on the similarity scores & pearson
    sim_scores_sort = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # # print(sim_scores_sort)
    # # Get the scores of the 10 most similar clothes
    sim_scores_top = sim_scores_sort[1:n+1]
    # print(sim_scores_top)
#     sep()

    #  # Get the clothes indices
    clothes_indices = [i[0] for i in sim_scores_top]
    res.append(clothes_indices)


[]

In [85]:
res

[[349, 3922, 7524, 5327, 4026],
 [9298, 5022, 5924, 4956, 8875],
 [5958, 944, 6715, 2332, 6701],
 [6476, 4446, 6603, 3346, 8391],
 [6491, 2906, 5065, 3423, 1743],
 [349, 3922, 7524, 5327, 4026],
 [9298, 5022, 5924, 4956, 8875],
 [5958, 944, 6715, 2332, 6701],
 [6476, 4446, 6603, 3346, 8391],
 [6491, 2906, 5065, 3423, 1743],
 [349, 3922, 7524, 5327, 4026],
 [9298, 5022, 5924, 4956, 8875],
 [5958, 944, 6715, 2332, 6701],
 [6476, 4446, 6603, 3346, 8391],
 [6491, 2906, 5065, 3423, 1743],
 [1753, 1928, 3637, 6646, 530],
 [4254, 2668, 649, 9284, 9200],
 [7270, 2529, 2324, 5415, 4024],
 [8512, 3825, 7656, 2780, 4709],
 [6002, 1817, 8585, 124, 4237],
 [5301, 7653, 4783, 3610, 3993],
 [627, 6649, 2175, 8977, 1320],
 [6528, 5732, 8152, 4210, 4212],
 [3506, 4142, 164, 4637, 5508],
 [6726, 8540, 1630, 3423, 6812],
 [818, 6366, 1932, 2281, 4469],
 [3984, 3078, 5246, 370, 5913],
 [5789, 2649, 8334, 4718, 3204],
 [3568, 8214, 1177, 8769, 8195],
 [7210, 3088, 7522, 1662, 8061],
 [9280, 6625, 9144, 480

In [ ]:
import recmetrics

pred = res


In [ ]:
recmetrics.personalization(predicted=pred)

# intra-list similarity

If a recommender system is recommending lists of very similar items to single users (for example, a user receives only recommendations of wedding clothes), then the intra-list similarity will be high.

In [ ]:

df_embed_ = pd.DataFrame(df_embed)
feature_df = df_embed_.iloc[:,:]

print(df_metadata.shape)
print(df_embed.shape)
print(feature_df.shape)

In [49]:
recmetrics.intra_list_similarity(pred, feature_df)

0.9972202526752296

In [50]:
# Dump df_metadata [PROSES EKSPOR]

print(df_metadata.columns)
df_metadata.head()


occasion = ('date','everyday','formal', 'other', 'party', 
            'vacation', 'wedding', 'work')
kat = (df_metadata.iloc[:,4:12])
kat = kat.values

fititem = ('fit', 'large', 'small')
fitcol = (df_metadata.iloc[:,12:15])
fitcol = fitcol.values

occlist = []
for row in kat:
    occ = []
    for j in range(len(row)):
        if row[j] != 0:
            occ.append(occasion[j])
    occlist.append(occ)
print(len(occlist))
df_metadata['occasion'] = occlist

fitlist = []
for row in fitcol:
    ft = []
    for j in range(len(row)):
        if row[j] != 0:
            ft.append(fititem[j])
    fitlist.append(ft)
print(len(fitlist))
df_metadata['fit_feedback'] = fitlist


df_metadata.to_pickle('items_metadata')

Index(['rating', 'height_cm', 'weight_kg', 'age_', 'date', 'everyday',
       'formal affair', 'other', 'party', 'vacation', 'wedding', 'work', 'fit',
       'large', 'small', 'vote_count', 'score', 'item_id_size', 'counter'],
      dtype='object')
9377
9377


In [52]:
# simulasi mengubah dataframe menjadi data

df_rekom_item = getItemInCounter(clothes_indices)
rekom_items = df_rekom_item.to_dict('records')
rekom_items

[{'rating': 10.0,
  'height_cm': 160.0,
  'weight_kg': 59.0,
  'age_': 25.0,
  'date': 0.0,
  'everyday': 0.0,
  'formal affair': 0.0,
  'other': 1.0,
  'party': 0.0,
  'vacation': 0.0,
  'wedding': 0.0,
  'work': 0.0,
  'fit': 0.0,
  'large': 1.0,
  'small': 0.0,
  'vote_count': 1,
  'score': 9.304591824716843,
  'item_id_size': '716072,16',
  'counter': 8374,
  'occasion': ['other'],
  'fit_feedback': ['large']},
 {'rating': 10.0,
  'height_cm': 160.0,
  'weight_kg': 59.0,
  'age_': 25.0,
  'date': 0.0,
  'everyday': 0.0,
  'formal affair': 1.0,
  'other': 0.0,
  'party': 0.0,
  'vacation': 0.0,
  'wedding': 0.0,
  'work': 0.0,
  'fit': 1.0,
  'large': 0.0,
  'small': 0.0,
  'vote_count': 1,
  'score': 9.304591824716843,
  'item_id_size': '1345027,8',
  'counter': 1670,
  'occasion': ['formal'],
  'fit_feedback': ['fit']},
 {'rating': 10.0,
  'height_cm': 160.0,
  'weight_kg': 59.0,
  'age_': 25.0,
  'date': 0.0,
  'everyday': 0.0,
  'formal affair': 0.0,
  'other': 0.0,
  'party': 0